In [ ]:
from __future__ import division

import os
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller


# test tationarity
def test_stationarity(timeseries):
    """
    Source:
    """
    
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=12)
    rolstd = pd.rolling_std(timeseries, window=12)

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print 'Results of Dickey-Fuller Test:'
    dftest = adfuller(timeseries.iloc[:,0].values, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print dfoutput


data = pd.read_csv('data.csv')

# Format datetime and extract year and month separately.
data['Date of Contact'] = pd.to_datetime(data['Date of Contact'], format='%Y-%m-%d')
data['month'] = data['Date of Contact'].dt.month
data['year'] = data['Date of Contact'].dt.year


# Total Contacts
df = data.groupby(['year','month'])['Client Name'].count().reset_index()

# convert to datetime
date_list = df['month'].astype(str) + "-" + df['year'].astype(str)
date_list = pd.to_datetime(date_list, format='%m-%Y')

df['index'] = date_list
df = df.set_index(['index'])

df = df[['Client Name']]
df.columns = [['count']]
df.index.name = None

#Plot and show
df['count'].plot(figsize=(12,8), title= 'Monthly Contacts', fontsize=14)
plt.show()

decomposition = seasonal_decompose(df['count'], freq=12)  
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(15, 8)


# Testing

# First Differencing
df['first_difference'] = df['count'] - df['count'].shift(1)


# Seasonal
df['seasonal_difference'] = df['count'] - df['count'].shift(12)  

